# Crypto 1 - Smart

> Only smart people can get the flag
>
> - mechfrog88
>
> Files: [main.py](main.py)

So I didn't actually manage to solve this challenge live, but the solution is semi-obvious in hindsight.

Roughly speaking, you are a given a function $f : \mathbb{Z}^4 \to \mathbb{Z}_{1657} \times \mathbb{Z}_{1663} \times \mathbb{Z}_{1049} \times \mathbb{Z}_{1193}$.

And then you also have a funcion $h(a,b,c,d) = f^k(a,b,c,d)$ where $k = 5555 + (abcd \bmod{3000000})$.

The challenge is that it generates you the value of $target := h(v_0,v_1,v_2,v_3)$ from some initial state whose values are random 32-bit ints, and you have to find values $(a,b,c,d) \in \mathbb{Z}_{1657} \times \mathbb{Z}_{1663} \times \mathbb{Z}_{1049} \times \mathbb{Z}_{1193}$ satisfying $h(a,b,c,d) = target$.

In particular, the original 32-bit state is (usually) not a valid solution, because the values tend to be larger than 4 digits each.

Anyway, the solve path seems obvious for the most part: if you can invert $f$ efficiently, then you can just keep doing this and checking if satisfies the $h$ condition. Now $f$ looks almost linear, but with a quadratic and a quartic term thrown into the mix. So I spent most of my time trying to exploit the near-linearity to invert it, but did not manage to do so.

Instead, the intended solution is to keep iterating $f$ _forwards_ until it hits a cycle. Since there's only roughly $2^{42}$ possible outcomes of $f$, the birthday paradox (or similar) basically states that if you iterate $O(2^{21})$ times there's a good chance your $f$ will hit a value you've already seen, and there's your cycle! Plus this has the added bonus of inverting all those states as well, not just one at a time.

Anyway, once you know this trick it's all trivial, so here's the solve script:

In [1]:
p = 1657; q = 1663; r = 1049; s = 1193

def f(v):
    a = (5 * v[0] + 3 * v[1] + 7 * v[2] + 4 * v[3] + v[0] * v[1] + 10 * (v[2] * v[3])**2) % p
    b = (9 * v[0] + 2 * v[1] + 1 * v[2] + 1 * v[3] + v[1] * v[2] + 11 * (v[0] * v[3])**2) % q
    c = (6 * v[0] + 7 * v[1] + 3 * v[2] + 9 * v[3] + v[2] * v[3] + 12 * (v[0] * v[1])**2) % r
    d = (8 * v[0] + 5 * v[1] + 2 * v[2] + 7 * v[3] + v[3] * v[0] + 13 * (v[1] * v[2])**2) % s
    return (a,b,c,d)

# fill this in with [target]
arr = [(379, 1641, 4, 1166)]

for _ in range(3000000):
    v = f(arr[-1])
    if v == arr[0]:
        print(f'found loop of length {len(arr)}')
        break
    arr.append(v)
else:
    assert False, 'did not find loop'
    
for i, v in enumerate(arr[::-1], 1):
    k = 5555 + ((v[0] * v[1] * v[2] * v[3]) % 3000000)
    if k % len(arr) == i:
        print(v)
        break
else:
    assert False, 'did not find soln'

found loop of length 1022119
(649, 652, 55, 296)


This doesn't succeed all the time, but it's not super-rare either, so we can do this manually (keep repeating until success):
1. Open a shell and connect to remote
2. Copy values here (into `arr` above)
3. Copy the solution back into the shell

The flag is `grey{hello_smart_admin_;D_hRkPxgxcMB7Yxk4e}`.